# Compute the NLL metric before and after tracking

In [1]:
import numpy as np
import torch
import math
import os
import properscoring as ps
import warnings
warnings.filterwarnings('ignore')

In [41]:
before_path = "../det/check/check_loss_two_step_center_sr_ind/"
after_path = "./sort/cov/"
after_cfk_path = after_path + "cfk/"
after_nlla_path = after_path + "nlla/"
after_origin_path = after_path + "origin/"
after_cfk_nlla_path = after_path + "cfk_nlla/"
after_cfk_wcp_path = after_path + "cfk_wcp/"
after_nlla_wcp_path = after_path + "nlla_wcp/"
after_cfk_nlla_wcp_path = after_path + "cfk_nlla_wcp/"
gt_path = "./TrackEval/data/gt/mot_challenge/"
scene_idxes_file_path = "../utils/test_scenes.txt"
var_cp_dict = {"upperbound": [125.44897552926587, 58.17835406418326, 224.21414415575381, 3907.8781994455294], "disco": [267.6059187072261, 65.37177933480551, 214.8165049926819, 4107.835669322297], "lowerbound": [339.5195216553872, 111.26186541465519, 298.26359427936194, 15076.497769642287], "colowerbound": [339.5195216553872, 111.26186541465519, 298.26359427936194, 15076.497769642287]}

In [37]:
def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
      [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
      the aspect ratio
    """
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w / 2.0
    y = bbox[1] + h / 2.0
    s = w * h  # scale is just area
    if float(h) < 0.00001:
        print([x, y, w, h])
        print(bbox)
    r = w / float(h)
    return np.array([x, y, s, r])


def convert_x_to_bbox(x, score=None):
    """
    Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
      [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
    """
    w = np.sqrt(x[2] * x[3])
    h = x[2] / w
    if score is None:
        return np.array(
            [x[0] - w / 2.0, x[1] - h / 2.0, x[0] + w / 2.0, x[1] + h / 2.0]
        ).reshape((1, 4))
    else:
        return np.array(
            [x[0] - w / 2.0, x[1] - h / 2.0, x[0] + w / 2.0, x[1] + h / 2.0, score]
        ).reshape((1, 5))

def split_data_by_frame(data):
    res = []
    start = 0
    if data.shape[0] == 0:
        return res
    if len(data.shape) == 1:
        res.append(data[np.newaxis, :])
        return res
    for i in range(1,data.shape[0]):
        if data[i][0] != data[start][0]:
            res.append(data[start:i,:])
            start = i
    res.append(data[start:,:])
    return res

def iou_batch(bb_test, bb_gt):
    """
    From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)

    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0.0, xx2 - xx1)
    h = np.maximum(0.0, yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
        + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1])
        - wh
    )
    return o

def linear_assignment(cost_matrix):
    try:
        import lap

        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i], i] for i in x if i >= 0])  #
    except ImportError:
        from scipy.optimize import linear_sum_assignment

        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

def pre_process_bbox(datas):
    """
    make the bbox be [x1,y1,x2,y2] where [x1,y1] is the left bottom and [x2,y2] is the right top
    """
    for i in range(datas.shape[0]):
        small = [min(datas[i][0], datas[i][2]), min(datas[i][1], datas[i][3])]
        large = [max(datas[i][0], datas[i][2]), max(datas[i][1], datas[i][3])]
        datas[i][0] = small[0]
        datas[i][1] = small[1]
        datas[i][2] = large[0]
        datas[i][3] = large[1]
    return datas

def remove_error_bbox(dets):
    res = []
    for i in range(dets.shape[0]):
        if dets[i, 0] == dets[i, 2] or dets[i, 1] == dets[i, 3]:
            continue
        res.append(list(dets[i]))
    return np.array(res)

def convert_bboxs_to_zs(datas):
    for i in range(datas.shape[0]):
        datas[i, :4] = convert_bbox_to_z(datas[i,:4])
    return datas
    
def compute_nll(dets, gts, mode, is_before_tracking, iou_threshold, is_use_cp_cov):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return [], matched_indices
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = torch.from_numpy(convert_bboxs_to_zs(matched_dets[:,:4]))[:,:2]
    target = torch.from_numpy(convert_bboxs_to_zs(matched_gts[:,:4]))[:,:2]
    if is_before_tracking:
        cov = np.exp(matched_dets[:,8:10])
    else:
        cov = matched_dets[:,8:10]
    if is_use_cp_cov:
        cp_cov = np.array(var_cp_dict[mode])[:2]
        cov = cov * cp_cov
    cov = torch.from_numpy(cov)
    std = torch.sqrt(cov)
    predicted_normal_dists = torch.distributions.normal.Normal(pred, std)
    negative_log_prob = - predicted_normal_dists.log_prob(target)
    negative_log_prob = torch.clamp(negative_log_prob, min = 0)
    negative_log_prob = torch.sum(negative_log_prob,axis=1)/2
    return negative_log_prob.tolist(), matched_indices

def compute_nll_for_mode(mode, det_path, is_before_tracking, iou_threshold, is_use_cp_cov):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    nll_res = []
    for agent in range(1,6):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-test" + str(agent)
        for scene in scene_idxes:
            if is_before_tracking:
                det_scene_file = os.path.join(det_files_path, "det_" + str(scene) + ".txt")
            else:
                det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            det_idx = 0
            for gt_idx in range(len(gt_datas)):
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    #print(f"Error idx not match {scene} {agent} {det_idx}, {gt_idx} {det_datas[det_idx][0][0]} {gt_datas[gt_idx][0][0]}")
                    continue
                if gt_idx >= len(gt_datas):
                    print(f"gt_idx out of range, {gt_idx} {det_idx}")
                    break
                nll, matched_indices = compute_nll(det_datas[det_idx], gt_datas[gt_idx], mode, is_before_tracking, iou_threshold, is_use_cp_cov)
                # if nll == []:
                #     print(det_scene_file)
                #     print(det_datas[det_idx])
                #     return 0
                nll_res.extend(nll)
                det_idx += 1
    nll_mean = sum(nll_res) / len(nll_res)
    return nll_mean

def show_one_result(path, is_before_tracking=True, iou_threshold = 0.5, is_use_cp_cov=True):
    mode_list = ["upperbound", "disco", "lowerbound", "colowerbound"]
    for mode in mode_list:
        nll = compute_nll_for_mode(mode, path, is_before_tracking, iou_threshold, is_use_cp_cov)
        print(f"NLL for {mode} is {nll}")

## before tracking

In [4]:
mode = "upperbound"
nll = compute_nll_for_mode(mode, before_path, True, 0.3, True)
print(f"NLL for {mode} is {nll}")

NLL for upperbound is 243.07201963561593


In [42]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 25.699231431559962
NLL for disco is 26.212783464459413
NLL for lowerbound is 25.17422176378463
NLL for colowerbound is 983.6787714744612


In [43]:
show_one_result(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 14.537769972912795
NLL for disco is 19.504103748329204
NLL for lowerbound is 13.03896916503081
NLL for colowerbound is 97.98319455448093


In [39]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 1940.6425392159501
NLL for disco is 2229.1614048727024
NLL for lowerbound is 3361.2403591519405
NLL for colowerbound is 128403.66567037892


In [10]:
show_one_result(before_path, True, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 969.446443360705
NLL for disco is 1487.1246234201192
NLL for lowerbound is 1671.877655285419
NLL for colowerbound is 12169.614310285411


In [11]:
var_cp_dict = {"upperbound": [10,10,10,10], "disco": [10,10,10,10], "lowerbound": [10,10,10,10], "colowerbound": [10,10,10,10]}
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 192.748649114817
NLL for disco is 221.62392301704477
NLL for lowerbound is 334.83144053199726
NLL for colowerbound is 12839.01345825878


In [12]:
show_one_result(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 95.62866152928744
NLL for disco is 147.4237153107055
NLL for lowerbound is 165.89159049758163
NLL for colowerbound is 1215.6163459946722


In [8]:
var_cp_dict = {"upperbound": [50,50,50,50], "disco": [50,50,50,50], "lowerbound": [50,50,50,50]}
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 38.06261961469876
NLL for disco is 43.85217717168636
NLL for lowerbound is 66.50840306297361


## after tracking

In [6]:
show_one_result(after_origin_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 1.2846157007495433
NLL for disco is 1.3193165236999298
NLL for lowerbound is 2.1516707394474954


In [7]:
show_one_result(after_origin_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 1.0115359920476263
NLL for disco is 1.2919162189065223
NLL for lowerbound is 1.6551647310742101


In [10]:
show_one_result(after_cfk_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 8.870087317394397
NLL for disco is 9.663970415801112
NLL for lowerbound is 13.706891949732503


In [4]:
show_one_result(after_cfk_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 7.300155854601104
NLL for disco is 9.096467606254752
NLL for lowerbound is 10.421989889732322


In [18]:
show_one_result(after_cfk_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 666.5098664018469
NLL for disco is 751.2329052216426
NLL for lowerbound is 1754.089095414952


In [19]:
show_one_result(after_cfk_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 522.185199100231
NLL for disco is 688.6418878705191
NLL for lowerbound is 1296.0360006671833


In [11]:
show_one_result(after_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 1.3420999143994659
NLL for disco is 1.4123933740918249
NLL for lowerbound is 2.19174335933121


In [5]:
show_one_result(after_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 1.0524389430094097
NLL for disco is 1.375625294048954
NLL for lowerbound is 1.7050658882616696


In [20]:
show_one_result(after_nlla_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 1.2846157007495433
NLL for disco is 1.3192193374501395
NLL for lowerbound is 2.1516707394474954


In [21]:
show_one_result(after_nlla_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 1.0115359920476263
NLL for disco is 1.291825456504012
NLL for lowerbound is 1.6551647310742101


In [13]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 9.60877227623219
NLL for disco is 12.094097605517934
NLL for lowerbound is 14.449547233541313
NLL for colowerbound is 720.0665852783403


In [14]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 7.869151841412263
NLL for disco is 11.208793168262304
NLL for lowerbound is 11.06317943727823
NLL for colowerbound is 58.71713269373099


In [22]:
show_one_result(after_cfk_nlla_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 666.6922176489993
NLL for disco is 751.1562228034281
NLL for lowerbound is 1754.089095414952


In [23]:
show_one_result(after_cfk_nlla_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 522.3708465695639
NLL for disco is 688.592218012732
NLL for lowerbound is 1296.0360006671833


In [45]:
#cp1
var_cp_dict = {"upperbound": [2.0168343618519997, 20.655256274811332, 1099429.4444401297, 0.008725584208192933], "disco": [1.5844876978208504, 16.85182149486315, 1199409.2165852713, 0.008524340686899256], "lowerbound": [0.9496513748504812, 14.405661414225111, 224888.44172695288, 0.007472273158473342]}
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 1.4053224996892848
NLL for disco is 1.2616917770704525
NLL for lowerbound is 2.46763601312665


In [46]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.8634192478850629
NLL for disco is 1.1276007523395488
NLL for lowerbound is 1.6027910087219057


In [15]:
#cp2
var_cp_dict = {"upperbound": [5.6827505661910545, 39.22419803102757, 2858468.784098897, 0.34036921505256784], "disco": [5.134614410746115, 38.66801996668446, 3366505.118310001, 0.3417140174552086], "lowerbound": [4.463060508876752, 30.744574219188294, 756935.3774669303, 0.12091569877559906], "colowerbound": [2.648966498640901, 10.699904022348633, 78644.8174770782, 0.14934529673210137]}
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.9405700552850172
NLL for disco is 0.9490738756867219
NLL for lowerbound is 1.295810844861161
NLL for colowerbound is 105.25310531615628


In [16]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.7402229653551322
NLL for disco is 0.9022929127629292
NLL for lowerbound is 1.0573760732673183
NLL for colowerbound is 8.130740079669824


# CRPS

In [44]:
var_cp_dict = {"upperbound": [125.44897552926587, 58.17835406418326, 224.21414415575381, 3907.8781994455294], "disco": [267.6059187072261, 65.37177933480551, 214.8165049926819, 4107.835669322297], "lowerbound": [339.5195216553872, 111.26186541465519, 298.26359427936194, 15076.497769642287], "colowerbound": [651.6848353663652, 353.78630002788134, 253.72976959856496, 41773.83374474704]}
def compute_crps(dets, gts, mode, is_before_tracking, iou_threshold, is_use_cp_cov):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return [], matched_indices
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = convert_bboxs_to_zs(matched_dets[:,:4])[:,:2]
    target = convert_bboxs_to_zs(matched_gts[:,:4])[:,:2]
    if is_before_tracking:
        cov = np.exp(matched_dets[:,8:10])
    else:
        cov = matched_dets[:,8:10]
    if is_use_cp_cov:
        cp_cov = np.array(var_cp_dict[mode])[:2]
        cov = cov * cp_cov
    std = np.sqrt(cov)
    crps = ps.crps_gaussian(target, 
                               pred,
                               std)
    return np.mean(crps, axis=1).tolist(), matched_indices

def compute_crps_for_mode(mode, det_path, is_before_tracking, iou_threshold, is_use_cp_cov):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    nll_res = []
    for agent in range(1,6):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-test" + str(agent)
        for scene in scene_idxes:
            if is_before_tracking:
                det_scene_file = os.path.join(det_files_path, "det_" + str(scene) + ".txt")
            else:
                det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            det_idx = 0
            for gt_idx in range(len(gt_datas)):
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    #print(f"Error idx not match {scene} {agent} {det_idx}, {gt_idx} {det_datas[det_idx][0][0]} {gt_datas[gt_idx][0][0]}")
                    continue
                if gt_idx >= len(gt_datas):
                    print(f"gt_idx out of range, {gt_idx} {det_idx}")
                    break
                nll, matched_indices = compute_crps(det_datas[det_idx], gt_datas[gt_idx], mode, is_before_tracking, iou_threshold, is_use_cp_cov)
                # if nll == []:
                #     print(det_scene_file)
                #     print(det_datas[det_idx])
                #     return 0
                nll_res.extend(nll)
                det_idx += 1
    nll_mean = sum(nll_res) / len(nll_res)
    return nll_mean

def show_one_result_crps(path, is_before_tracking=True, iou_threshold = 0.5, is_use_cp_cov=True):
    mode_list = ["upperbound","disco", "lowerbound", "colowerbound"]
    for mode in mode_list:
        nll = compute_crps_for_mode(mode, path, is_before_tracking, iou_threshold, is_use_cp_cov)
        print(f"NLL for {mode} is {nll}")

In [34]:
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 0.47186133004112474
NLL for disco is 0.5157464671370288
NLL for lowerbound is 0.7103145869770786
NLL for colowerbound is 0.7272603059282656


In [35]:
show_one_result_crps(before_path, True, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 0.41094683792914216
NLL for disco is 0.4766803297192767
NLL for lowerbound is 0.5307394859097074
NLL for colowerbound is 0.468651299114203


In [51]:
var_cp_dict = {"upperbound": [125.44897552926587, 58.17835406418326, 224.21414415575381, 3907.8781994455294], "disco": [267.6059187072261, 65.37177933480551, 214.8165049926819, 4107.835669322297], "lowerbound": [339.5195216553872, 111.26186541465519, 298.26359427936194, 15076.497769642287], "colowerbound": [51.6848353663652, 53.78630002788134, 53.72976959856496, 1773.83374474704]}
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.42404725439307944
NLL for disco is 0.4655606713319821
NLL for lowerbound is 0.652130084266047
NLL for colowerbound is 0.6941032807006245


In [52]:
show_one_result_crps(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.3637884493068222
NLL for disco is 0.42720285262171986
NLL for lowerbound is 0.4751655746478185
NLL for colowerbound is 0.4363331266962258


In [9]:
show_one_result_crps(after_origin_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.3641422764569029
NLL for upperbound is 0.33362799778691216
NLL for lowerbound is 0.4894665942261575


In [10]:
show_one_result_crps(after_origin_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.35962919751963884
NLL for upperbound is 0.32065571890555494
NLL for lowerbound is 0.43649402973124446


In [11]:
show_one_result_crps(after_cfk_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.33004108293047013
NLL for upperbound is 0.2985007779702155
NLL for lowerbound is 0.450105484839003


In [12]:
show_one_result_crps(after_cfk_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.3247238723081699
NLL for upperbound is 0.2852119596264233
NLL for lowerbound is 0.39717911938372136


In [25]:
show_one_result_crps(after_cfk_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.37586829772617725
NLL for upperbound is 0.3454551756523389
NLL for lowerbound is 0.49989039831295196


In [26]:
show_one_result_crps(after_cfk_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.36973119658777187
NLL for upperbound is 0.3314054357546768
NLL for lowerbound is 0.44620831357182467


In [13]:
show_one_result_crps(after_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.3897910772927842
NLL for upperbound is 0.3512990329277939
NLL for lowerbound is 0.5062101621863176


In [14]:
show_one_result_crps(after_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.38327219770601834
NLL for upperbound is 0.33613366313168286
NLL for lowerbound is 0.45363809511688


In [27]:
show_one_result_crps(after_nlla_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.3641346002493097
NLL for upperbound is 0.33362799778691216
NLL for lowerbound is 0.4894665942261575


In [28]:
show_one_result_crps(after_nlla_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.35962272316889227
NLL for upperbound is 0.32065571890555494
NLL for lowerbound is 0.43649402973124446


In [28]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 0.3123616378377446
NLL for disco is 0.3545389467390727
NLL for lowerbound is 0.4629444218258531
NLL for colowerbound is 0.7638918204825628


In [29]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 0.29705635324554813
NLL for disco is 0.3470856569834966
NLL for lowerbound is 0.40932565656219966
NLL for colowerbound is 0.4268081456980423


In [ ]:
show_one_result_crps(after_cfk_nlla_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

In [30]:
show_one_result_crps(after_cfk_nlla_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.36977329721258473
NLL for upperbound is 0.33144440749524273
NLL for lowerbound is 0.44620831357182467


In [63]:
#cp1
var_cp_dict = {"upperbound": [2.0168343618519997, 20.655256274811332, 1099429.4444401297, 0.008725584208192933], "disco": [1.5844876978208504, 16.85182149486315, 1199409.2165852713, 0.008524340686899256], "lowerbound": [0.9496513748504812, 14.405661414225111, 224888.44172695288, 0.007472273158473342], 'colowerbound': [0.417313590703244, 3.079141799058277, 16624.535995030397, 0.00040261623152204926]}
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.3011598967603216
NLL for disco is 0.33606329218423914
NLL for lowerbound is 0.44416975378327334
NLL for colowerbound is 0.7550522959445382


In [64]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.28606787335712
NLL for disco is 0.3289714848899278
NLL for lowerbound is 0.3917155607592708
NLL for colowerbound is 0.4182583957522243


In [55]:
#cp2
var_cp_dict = {"upperbound": [5.6827505661910545, 39.22419803102757, 2858468.784098897, 0.34036921505256784], "disco": [5.134614410746115, 38.66801996668446, 3366505.118310001, 0.3417140174552086], "lowerbound": [4.463060508876752, 30.744574219188294, 756935.3774669303, 0.12091569877559906], "colowerbound": [2.648966498640901, 10.699904022348633, 78644.8174770782, 0.14934529673210137], "colowerbound": [2.648966498640901, 10.699904022348633, 78644.8174770782, 0.14934529673210137]}
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.32709907057423476
NLL for disco is 0.3668941119070588
NLL for lowerbound is 0.47195265159244754
NLL for colowerbound is 0.7751045365367206


In [56]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.3119193496780249
NLL for disco is 0.36016519597971186
NLL for lowerbound is 0.4210904762041204
NLL for colowerbound is 0.4428183691763729


In [23]:
var_cp_dict = {"upperbound": [10,10,10,10], "disco": [10,10,10,10], "lowerbound": [10,10,10,10], "colowerbound": [10,10,10,10]}
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.45306145631208883
NLL for disco is 0.49805945284303266
NLL for lowerbound is 0.6931012107098485
NLL for colowerbound is 0.7133724332271444


In [24]:
show_one_result_crps(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.39236181264625586
NLL for disco is 0.45911130432855074
NLL for lowerbound is 0.5139045873088862
NLL for colowerbound is 0.45504230290317543
